# EDA for Location Puzzle

In [1]:
import pandas as pd

# The locations puzzle
Expedia presented us with a dataset where countries and cities are hidden behind integer codes. Is it possible to find out which city is which?

Read in a few lines to get a list of columns.

In [2]:
train = pd.read_csv(r"C:\Users\Ankur\Desktop\CSC\train.csv", nrows=10)
train.columns

Index(['date_time', 'site_name', 'posa_continent', 'user_location_country',
       'user_location_region', 'user_location_city',
       'orig_destination_distance', 'user_id', 'is_mobile', 'is_package',
       'channel', 'srch_ci', 'srch_co', 'srch_adults_cnt', 'srch_children_cnt',
       'srch_rm_cnt', 'srch_destination_id', 'srch_destination_type_id',
       'is_booking', 'cnt', 'hotel_continent', 'hotel_country', 'hotel_market',
       'hotel_cluster'],
      dtype='object')

#Columns related to user location are:
  user_location_country
  user_location_region
  user_location_city
Columns related to hotel location are:
  hotel_country
  hotel_market
  srch_destination_id
Finally, the orig_destination_distance column shows us the distance in miles between the user and their chosen hotel.
We should note that hotel countries and user location countries are encoded differently, meaning that the same country will have different numbers in these two columns. I will not go into srch_destination_ids yet because they might represent different locations within the same city so this division is probably too fine. On the other hand the hotel_markets correspond to nonoverlapping regions all over the globe, and large cities are covered by their own hotel_markets so this is a nice match to user_location_city column.
Now let's read in a million rows using only columns relating to this task. Drop rows where distance is undefined.

In [3]:
train = pd.read_csv(r"C:\Users\Ankur\Desktop\CSC\train.csv", usecols = ['posa_continent', 
       'user_location_country',
       'user_location_region', 'user_location_city',
       'orig_destination_distance','hotel_continent', 
       'hotel_country', 'hotel_market'], nrows=1000000).dropna()

Now, a mapping of user and hotel countries- 
If a user books a hotel in their own city then we should see a very short distance in the corresponding dataset row. So we can look at which user and hotel countries have the lowest minimum distances between them and deduce that these pairs probably refer to the same actual country.
Let's group rows by user country and hotel country and look at the distances.

In [4]:
distaggs = (train.groupby(['user_location_country','hotel_country'])
            ['orig_destination_distance']
            .agg(['min','mean','max','count']))
distaggs.sort_values(by='min').head(20)

,,min,mean,max,count
user_location_country,hotel_country,,,,
66,50,0.0056,860.307373,5156.8218,323353
205,198,0.0056,484.961579,3113.8813,20790
46,144,0.0060,197.404405,500.7198,2178
1,105,0.0766,222.188807,730.5410,3754
77,63,0.0793,661.430207,2319.4878,4228
63,46,0.1388,53.162275,146.8886,114
16,112,0.1518,21.953550,90.7257,18
182,196,0.1719,495.666961,1892.6119,654
215,8,0.1894,455.140957,1829.7483,2369


So, we see a huge number of rows belonging to user country 66 and hotel country 50. It's probably the USA. Then there are some more pairs with low distances.
First repeated row is user country 205 and hotel country 50 again. And the minimum distance is 3 miles here - larger than 0.0056 we saw in the first rows. So user country 205 must be some neighbor country, Canada or Mexico.
Then there's the repeat appearance of user country 66 with travels to hotel countries 8 and 198 - those are probably again Canada and Mexico.
By the end of the table shown minimum distances go up to almost 45 miles, so this criterion does not work so obviously any more - the pairs might be just neighboring countries, and not necessarily the same one.

# user_location_country 66

How many regions does this country have?

In [5]:
c66 = train[train.user_location_country==66]
c66.user_location_region.unique().shape

(51,)

-51 looks fitting for the USA.

Let's look at trips within this country.
The USA have Hawaii which is a popular tourist location and also very far away from other regions. I'll group the data by user_location_region and hotel_market and take a look at maximum distances.

In [6]:
c66in = c66[c66.hotel_country==50]
(c66in.groupby(['user_location_region','hotel_market'])['orig_destination_distance']
      .agg(['min','mean','max','count'])
      .sort_values(by='max',ascending=False).head(20))

min         mean        max  count
user_location_region hotel_market                                          
315                  212           5095.9786  5105.605659  5156.8218     17
311                  212           5056.8921  5080.972996  5153.1368    110
392                  214           5140.6513  5142.062586  5148.4290      7
246                  691           5148.0152  5148.015200  5148.0152      1
315                  636           5062.8436  5128.082768  5145.4430     25
311                  214           5143.3044  5143.941000  5144.2593      3
                     636           5000.9822  5041.348265  5107.5297     34
                     213           5045.0593  5068.213186  5096.3858      7
335                  214           5095.5180  5095.518000  5095.5180      1
246                  1538          5092.1863  5092.186300  5092.1863      1
                     690           5085.5615  5086.200625  5086.9340      8
392                  212           5075.6274  5083.410963  5085.7999     19
189                  214           5023.8472  5039.981190  5072.0866     30
                     212           4982.7392  5013.195314  5063.9225     36
335                  212           5046.4997  5046.560000  5046.6203      2
392                  213           5042.9305  5042.930500  5042.9305      1
348                  214           5005.4468  5020.192196  5041.3712     27
                     212           4693.8853  4957.907170  5028.1678    293
337                  214           4980.6657  5006.983213  5020.8658     30
246                  676           4965.5305  5001.341133  5019.6610      3

Looks like we have a lot of hotel_market values 212, 214 and a couple of 213 for good measure. These could be our paradise islands.

Let's look at distances from hotel_market 212 to different user cities in the USA sorting by popularity.

In [10]:
hawaii = (c66in[c66in.hotel_market == 212]
          .groupby(['user_location_region','user_location_city'])
          ['orig_destination_distance']
          .agg(['min','mean','max','count'])
          .sort_values(by='count',ascending=False))
hawaii.head(10)

min         mean        max  \
user_location_region user_location_city                                      
174                  24103               2553.6476  2558.920339  2572.9294   
                     26232               2388.9474  2398.424736  2410.4776   
348                  48862               4955.8994  4965.165163  4977.7217   
246                  50661                  0.0961     1.262481    30.8003   
174                  16634               2406.4423  2417.656113  2427.6747   
442                  35390               3892.9132  3893.785301  3894.3728   
174                  25317               2576.2162  2583.085756  2597.1833   
462                  27117               2690.7916  2692.769746  2701.2926   
174                  9890                2577.8801  2581.649199  2595.6242   
                     17494               2401.3502  2410.932780  2422.6211   

                                         count  
user_location_region user_location_city         
174                  24103                 225  
                     26232                 205  
348                  48862                 130  
246                  50661                 122  
174                  16634                 112  
442                  35390                 106  
174                  25317                 102  
462                  27117                  99  
174                  9890                   93  
                     17494                  84

Looks like we caught some very local trips in row 4. So region 246 is probably Hawaii.

The site http://www.distancefromto.net/ tells us that distances from Honolulu to other cities are:
          San Francisco - 2397.40 miles (the second line probably)
          Los Angeles - 2562.87 miles (could be the first line)
          New York - 4965.20 miles (could be the third)
So region 174 must be California and 348 New York with 48862 being New York city.


# Let's look at trips from New York city.

In [8]:
fromny = (c66in[(c66in.user_location_region == 348) & 
                (c66in.user_location_city == 48862)]
          .groupby(['hotel_market'])
          ['orig_destination_distance']
          .agg(['min','mean','max','count'])
          .sort_values(by='count',ascending=False))
fromny.head(10)

,min,mean,max,count
hotel_market,,,,
675,0.0584,1.137781,4.9885,900
701,1082.6153,1093.870519,1122.2943,764
628,2226.5168,2233.679107,2241.2550,451
1230,2566.7861,2568.553204,2571.9729,450
365,2433.4878,2455.587006,2474.3559,397
682,922.7518,956.316762,967.7378,315
191,188.9454,208.279871,223.8952,296
637,711.7014,713.956945,731.6175,257
623,72.0639,98.612501,106.8603,239


We can see that New York city itself is probably hotel_market 675.

Distances from New York:
         to Miami - 1093.57 miles -> hotel_market 701
         to Las Vegas - 2230.03 miles -> hotel_market 628
         to Los Angeles - 2448.30 miles -> hotel_market 365
         to San Francisco - 2568.57 miles -> hotel_market 1230
         to Chicago - 711.83 miles -> Chicago is hotel_market 637
         to Washington - 203.78 miles -> Washington might be hotel_market 191 
         to Philadelphia - 80.63 miles -> hotel_market 623 
         
We already know Los Angeles as a user_location_city. Let's do a check by confirming that trips from that city id to hotel market 365 have small distances.

In [9]:
(c66in[(c66in.hotel_market==365) & 
       (c66in.user_location_region==174) & 
       (c66in.user_location_city==24103)]
 ['orig_destination_distance'].describe())

count    806.000000
mean      13.620217
std       12.253812
min        0.148000
25%        5.741750
50%        9.510500
75%       16.453375
max       60.011400
Name: orig_destination_distance, dtype: float64

This looks about right!


# Results
It looks like it's completely possible to deanonymize the countries and cities in this dataset. At least the popular ones. The more countries and cities we identify the easier the subsequent task becomes. We could sort of triangulate the locations yet uncovered using distances to already known locations.